In [173]:
import enum
import typing
from dataclasses import dataclass,field


In [107]:
class Pulse(enum.Enum):
    Low = 0
    High = 1


In [142]:

@dataclass
class Module:
    name : str
    outputs : list[str]
    lowpulses :int = 0
    highpulses :int = 0

    def sendpulse(self,pulse,network):
        match pulse:
            case Pulse.Low:
                self.lowpulses +=len(self.outputs)
            case Pulse.High:
                self.highpulses +=len(self.outputs)
        for module in self.outputs:
            # print(self.name, pulse, module)
            network.pulsequeue.append((pulse,self.name,module))
@dataclass
class TheButton(Module):
    def __init__(self):
        self.name = "button"
        self.outputs = ["broadcaster"]

    def pressbutton(self,network):
        self.sendpulse(Pulse.Low,network)
        
@dataclass
class Broadcaster(Module):
    def receivepulse(self,pulse,name,network):
        self.sendpulse(pulse,network)

@dataclass
class FlipFlop(Module):
    status : bool = False
    def receivepulse(self,pulse,name,network):   
        if pulse==Pulse.Low:
            self.status = not self.status
            self.sendpulse(Pulse(self.status),network)

@dataclass
class Output(Module):
    def receivepulse(self,pulse,name,network):
        match pulse:
            case Pulse.Low:
                self.lowpulses += 1
            case Pulse.High:
                self.highpulses += 1

@dataclass
class Conjunction(Module):
    inputs : dict = field(default_factory=dict)
    def findinputs(self,network):
        for name,module in network.items():
            if self.name in module.outputs:
                self.inputs[name] = Pulse.Low
    
    def receivepulse(self,pulse,name,network):
        self.inputs[name] = pulse
        if Pulse.Low in self.inputs.values():
            self.sendpulse(Pulse.High,network)
        else:
            self.sendpulse(Pulse.Low,network)



In [172]:

@dataclass
class Network(dict):
    pulsequeue : list = field(default_factory=list)

    @classmethod
    def fromfile(cls,filename):
        network =cls()
        network["button"] =  TheButton()
        with open(filename) as f:
            for line in f:
                input,output = line.rstrip().split(" -> ")
                output = output.split(", ")
                match input[0]:
                    case "%":
                        network[input[1:]] = FlipFlop(input[1:],output)
                    case "&":
                        network[input[1:]] = Conjunction(input[1:],output)
                    case "b":
                        network["broadcaster"] = Broadcaster("broadcaster",output)
        for module in network.values():
            if type(module) is Conjunction:
                module.findinputs(network)
        return network
    
    def isinitiliazed(self):
        if self.pulsequeue: return False
        for module in self.values():
            if type(module) is Conjunction:
                if Pulse.High in module.inputs.values(): return False
            elif type(module) is FlipFlop:
                if module.status: return False
        return True

    def reset(self):
        self.pulsequeue = []
        for module in self.values():
            module.lowpulses = 0
            module.highpulses = 0
            if type(module) is Conjunction:
                for k in module.inputs.keys():
                    module.inputs[k] =Pulse.Low
            elif type(module) is FlipFlop:
                module.status = False
        return True

    def countpulses(self):
        low,high = 0,0
        for module in self.values():
            low += module.lowpulses
            high += module.highpulses
        return low*high, low, high
    
    def pressbutton(self):
        self["button"].pressbutton(self)
        self.processpulses()
    
    def processpulses(self):
        while self.pulsequeue:
            # print(len(self.pulsequeue))
            pulse, sender, receiver = self.pulsequeue.pop(0)
            if receiver in self.keys():
                self[receiver].receivepulse(pulse,sender,self)


# Part 1

In [149]:
network = Network.fromfile("input20.txt")
network["rx"] = Output("rx",[])

for _ in range(1000): network.pressbutton()
network.countpulses()

(797905680, 16656, 47905)

In [150]:
network["rx"] 

Output(name='rx', outputs=[], lowpulses=0, highpulses=5125)

In [138]:
network = Network.fromfile("ex20-2.txt")
network.pressbutton()
network.countpulses()

button Pulse.Low broadcaster
broadcaster Pulse.Low a
a Pulse.High inv
a Pulse.High con
inv Pulse.Low b
con Pulse.High output
b Pulse.High con
con Pulse.Low output


(16, 4, 4)

# Part 2

In [151]:
network = Network.fromfile("input20.txt")
network["rx"] = Output("rx",[])
numpress = 0
while network["rx"].lowpulses==0:
    network.pressbutton()
    numpress+=1
network.countpulses()
#Too long

KeyboardInterrupt: 

In [163]:
network = Network.fromfile("input20.txt")
network["rx"] = Output("rx",[])


In [170]:
network = Network.fromfile("input20.txt")
network["rx"] = Output("rx",[])
subcycles = {}
for mod in network["vr"].inputs.keys():
    network.reset()
    numpress = 0
    while network[mod].highpulses==0: 
        network.pressbutton()
        numpress+=1
    subcycles[mod] = numpress

In [171]:
subcycles

{'pq': (4001, False),
 'fg': (3929, False),
 'dk': (3793, False),
 'fm': (4007, False)}

In [169]:
4001*3929*3793*4007

238920142622879